<a href="https://colab.research.google.com/github/indmind/2141720003-machine-learning-2023/blob/main/tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 2**

In [5]:
import tensorflow as tf
import numpy as np
import os
import time

In [6]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [7]:
# Membaca teks dari file menggunakan mode 'rb' (binary mode) dan mendekode dengan encoding 'utf-8'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Panjang teks adalah jumlah karakter dalam teks tersebut
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [8]:
# Mencetak 250 karakter pertama dalam teks
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [9]:
# Mengidentifikasi karakter-karakter unik dalam teks
vocab = sorted(set(text))

# Mencetak jumlah karakter unik
print(f'{len(vocab)} unique characters')

65 unique characters


In [10]:
# Daftar teks contoh
example_texts = ['abcdefg', 'xyz']

# Memecah teks menjadi karakter-karakter Unicode
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasil karakter-karakter Unicode
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer:

In [11]:
# Membuat lapisan StringLookup untuk mengonversi karakter menjadi ID numerik
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),  # Daftar karakter-karakter yang ingin diindeks
    mask_token=None  # Token masking (jika ada), dalam hal ini, tidak ada masking
)

perintah diatas mengconvert token menjadi id

In [12]:
# Mengonversi karakter-karakter Unicode menjadi ID numerik
ids = ids_from_chars(chars)

# Menampilkan hasil ID numerik
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode
`tf.keras.layers.StringLookup(..., invert=True)`.

In [13]:
# Membuat lapisan StringLookup untuk mengonversi ID numerik ke karakter-karakter Unicode
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),  # Menggunakan vocabulary yang telah diindeks sebelumnya
    invert=True,  # Mengatur invert ke True untuk mengonversi kembali dari ID ke karakter
    mask_token=None  # Token masking (jika ada), dalam hal ini, tidak ada masking
)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter `tf.RaggedTensor`:

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [19]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [21]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Untuk pelatihan, Anda memerlukan kumpulan data pasangan (input, label). Dimana input dan label merupakan urutan. Pada setiap langkah waktu, inputnya adalah karakter saat ini dan labelnya adalah karakter berikutnya. Berikut adalah fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu:

In [22]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### **Membuat Batch Training**

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [26]:
# Batch size (ukuran batch) yang digunakan selama pelatihan
BATCH_SIZE = 64

# Buffer size (ukuran buffer) untuk mengacak urutan dataset
# TensorFlow data dirancang untuk bekerja dengan urutan yang mungkin tak terbatas,
# sehingga tidak mencoba untuk mengacak seluruh urutan di dalam memori.
# Sebaliknya, ia mempertahankan buffer di mana ia mengacak elemen.
BUFFER_SIZE = 10000

# Mengonfigurasi dataset dengan mengacak urutan, mengatur ukuran batch,
# dan menggunakan prefetch untuk optimalisasi
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)  # Mengacak urutan dataset
    .batch(BATCH_SIZE, drop_remainder=True)  # Mengatur ukuran batch dengan menghapus sisa data yang tidak cukup untuk satu batch
    .prefetch(tf.data.experimental.AUTOTUNE)  # Menggunakan prefetch untuk optimalisasi
)

# Menampilkan dataset yang telah dikonfigurasi
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Buat Model**

In [27]:
# Jumlah kata dalam vocabulary pada lapisan StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensi embedding
embedding_dim = 256

# Jumlah unit RNN (Recurrent Neural Network)
rnn_units = 1024

In [28]:
# Mendefinisikan kelas model khusus MyModel
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # Lapisan embedding untuk mengonversi ID numerik menjadi vektor embedding
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # Lapisan GRU (Gated Recurrent Unit) dengan return_sequences dan return_state
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    # Lapisan dense (sepenuhnya terhubung) dengan vocab_size output
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    # Menggunakan lapisan embedding
    x = self.embedding(x, training=training)

    if states is None:
      # Mendapatkan initial_state dari lapisan GRU jika states adalah None
      states = self.gru.get_initial_state(x)

    # Melakukan langkah propagasi pada lapisan GRU
    x, states = self.gru(x, initial_state=states, training=training)

    # Melakukan langkah propagasi pada lapisan dense
    x = self.dense(x, training=training)

    if return_state:
      # Mengembalikan output dan states jika return_state adalah True
      return x, states
    else:
      # Mengembalikan hanya output jika return_state adalah False
      return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,  # Jumlah kata dalam vocabulary
    embedding_dim=embedding_dim,  # Dimensi embedding
    rnn_units=rnn_units  # Jumlah unit dalam lapisan GRU
)

### **Uji Model**

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices= tf.squeeze(sampled_indices, axis=-1).numpy()

In [33]:
sampled_indices

array([34,  2, 11, 31,  8,  2, 22,  8, 49, 18,  6, 20, 19,  7, 55, 50, 12,
       57, 28, 14,  8, 23, 18, 18, 37, 29, 60, 24, 41, 10,  6, 56, 65,  9,
        7, 57, 24, 11, 52, 15, 63,  8, 58, 48, 48, 44, 32, 45, 24, 51,  2,
       34, 50, 26, 61, 36, 24, 24, 62, 18, 56,  4, 62, 28, 20, 24, 59, 10,
       37,  2, 36,  4, 39, 21, 45, 47, 14,  2, 45, 39, 52, 20, 49, 14, 46,
       27, 49, 64, 21, 49, 19, 38, 27, 44,  4, 17, 22, 33, 51, 32])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b", thus accused it:\nThat only like a gulf it did remain\nI' the midst o' the body, idle and unactive,\n"

Next Char Predictions:
 b"U :R- I-jE'GF,pk;rOA-JEEXPuKb3'qz.,rK:mBx-siieSfKl UkMvWKKwEq$wOGKt3X W$ZHfhA fZmGjAgNjyHjFYNe$DITlS"


### **Train Model**

### **Tambahan optimizer dan fungsi loss**

loss function `tf.keras.losses.sparse_categorical_crossentropy` standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag `from_logits`.

In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1912813, shape=(), dtype=float32)


Model yang baru diinisialisasi tidak boleh terlalu yakin dengan dirinya sendiri, semua log keluaran harus memiliki besaran yang sama. Untuk mengonfirmasi hal ini, Anda dapat memeriksa bahwa eksponensial dari loss rata-rata harus kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

66.107445

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [38]:
model.compile(optimizer='adam', loss=loss)

### **Konfigurasi Checkpoints**

Gunakan `tf.keras.callbacks.ModelCheckpoint` untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### **Lakukan Proses Training**

In [40]:
EPOCHS = 10

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 14s 54ms/step - loss: 2.7011
Epoch 2/10
172/172 [==============================] - 11s 53ms/step - loss: 1.9792
Epoch 3/10
172/172 [==============================] - 11s 54ms/step - loss: 1.6993
Epoch 4/10
172/172 [==============================] - 11s 54ms/step - loss: 1.5406
Epoch 5/10
172/172 [==============================] - 12s 56ms/step - loss: 1.4438
Epoch 6/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3766
Epoch 7/10
172/172 [==============================] - 12s 58ms/step - loss: 1.3235
Epoch 8/10
172/172 [==============================] - 12s 59ms/step - loss: 1.2794
Epoch 9/10
172/172 [==============================] - 12s 61ms/step - loss: 1.2372
Epoch 10/10
172/172 [==============================] - 13s 62ms/step - loss: 1.1978


### **Generate Teks**

Berikut ini membuat prediksi satu langkah:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
How is three news?

Provost:
How most awsicious cripiness' nor and bloody
To same your lordages the doom; Catesby:
Strike up in his pillaiment: therefore, so,
Then but so well his punish propity, hell, buy learning such a
cause. thou wouldst thou, grazegly!
Year, my grazed mother much.
His swafe, to the laws.

RICHARD:
Nay, he had noted both you pale as language,
Let's have by this stoopport suddemank. Now, Clifford,
Look in the first of at as pire to pleasing.

BISHOP OF CARLISLE:
Thou did it? You of all things all:
Therefore at reason isleep, that was hootlock and as being cannot
While Lewis for as at thought it makes.

RIVERS:
Hoars Prince, you are remainded in talkeh;
Old Kate, by Gid, 'for he stay than calms him her in hip vow?
No, Norfel, powers us for me. For thy now I did.

QUEEN RATER:
Surrey, About o'er his highness of my censure,
Which not used him.

PROSPERO:
Ay, if my suit: dishing me would have paids of door
And in their lamentation.

FRIAR JOHN:
I mean, my lord, y

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO: the present prince of hath some hearts to fault.\n\nPETRUCHIO:\nResice the mark that of my unrequerness death\nIs bedis; two despite, sure, let it; 'twas a meddlers.\n\nANGELO:\nCome to the Warwick, have any leave?\n\nANTONIO:\nTell him with greater to be; for they resort it, and\nBy name. God sir, if Kiss Gidrom prevosion.\nOr else my ancient shapes of me in blood,\nThat will were the lasque than I should find.\nCay she hath, good from Rome as thought and fetch'd\nindeed kiss in thine own eyes, and that she there no puts\nBitter to the king's subbloces, him, you have\ngentorsign and lawfully ligst making\nTebles of his necks? ORWARD IV:\nGust weep, and for this world,\nGod--knick are three and his sun beastsion of his king.\nHad should cumbs swear it out, 'tis mine:\nWho father what i' the lies tongues\nHave kill'd me your fitches, sir, and but sweet'st friendly,\nTo be the sky, om bittorke grows some broble time,\nHortension of a gracious tears never-finded,\nI l

### **Ekspor Model Generator**

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [47]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
O miserable!

QUEEN ELIZABETH:
Come, the pitient of the matches too poor dear;
I prityed go duke on


# **Praktikum**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan `tf.GradientTape` untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca `eager execution guide`.

Prosedurnya adalah:
1. Jalankan Model dan hitung loss dengan `tf.GradientTape`.
2. Hitung update dan terapkan pada model dengan optimizer

In [48]:
class CustomTraining(MyModel):
 @tf.function
 def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
        predictions = self(inputs, training=True)
        loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [49]:
model = CustomTraining(
     vocab_size=len(ids_from_chars.get_vocabulary()),
     embedding_dim=embedding_dim,
     rnn_units=rnn_units)

In [50]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [51]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 59ms/step - loss: 2.7391


In [52]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
      logs = model.train_step([inp, target])
      mean.update_state(logs['loss'])

      if batch_n % 50 == 0:
         template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
         print(template)

 # saving (checkpoint) the model every 5 epochs
      if (epoch + 1) % 5 == 0:
         model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1868
Epoch 1 Batch 50 Loss 2.0835
Epoch 1 Batch 100 Loss 2.0028
Epoch 1 Batch 150 Loss 1.8626

Epoch 1 Loss: 2.0059
Time taken for 1 epoch 13.86 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8342
Epoch 2 Batch 50 Loss 1.7534
Epoch 2 Batch 100 Loss 1.6716
Epoch 2 Batch 150 Loss 1.6631

Epoch 2 Loss: 1.7238
Time taken for 1 epoch 20.48 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6145
Epoch 3 Batch 50 Loss 1.6119
Epoch 3 Batch 100 Loss 1.5056
Epoch 3 Batch 150 Loss 1.5226

Epoch 3 Loss: 1.5553
Time taken for 1 epoch 11.49 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4745
Epoch 4 Batch 50 Loss 1.4416
Epoch 4 Batch 100 Loss 1.4486
Epoch 4 Batch 150 Loss 1.4265

Epoch 4 Loss: 1.4532
Time taken for 1 epoch 11.46 sec
_____________________________________________________________________

**Perbedaan dengan praktikum 2?**

Dalam praktikum 2, digunakan metode pelatihan yang lebih umum dan sederhana, yaitu model.fit, yang telah terintegrasi dengan TensorFlow. Metode ini secara otomatis mengelola banyak aspek pelatihan, termasuk perhitungan loss, perhitungan gradien, dan pembaruan bobot model.

Di sisi lain, dalam tugas praktikum yang diberikan, digunakan pendekatan pelatihan yang lebih spesifik dan kompleks. Dalam pendekatan ini, ada penggunaan metode train_step yang didefinisikan secara eksplisit dalam turunan model. Metode ini mengendalikan perhitungan loss, perhitungan gradien, dan pembaruan bobot model dengan jelas. Selain itu, ada penggunaan objek tf.metrics.Mean untuk menghitung rata-rata loss selama pelatihan. Pendekatan ini memberikan tingkat kontrol yang lebih tinggi dan lebih banyak opsi kustomisasi dalam proses pelatihan model.

Secara keseluruhan, perbedaan utama antara kedua pendekatan ini terletak pada tingkat kontrol yang lebih tinggi dan lebih banyak fleksibilitas yang diberikan oleh kode dalam tugas praktikum. Hal ini sangat bermanfaat ketika Anda perlu menyesuaikan pelatihan model secara khusus untuk tugas-tugas tertentu.